In [11]:
def return_test_frame(zone, row_num = 50000):
    #Import
    import pickle
    import mysql.connector
    import numpy as np
    import pandas as pd
    from pandas import DataFrame
    from sklearn.preprocessing import StandardScaler
    from sklearn.svm import SVR
    import mysql.connector
    import os
    import requests as r
    from sklearn import metrics
    
    
    #Load models
    target_scaler = 'tar_scaler_{0}.sav'.format(zone)
    feature_scaler = 'feat_scaler_{0}.sav'.format(zone)
    model = "model_zone_{0}.sav".format(zone)

    tar_scaler = pickle.load(open(target_scaler, 'rb'))
    feat_scaler = pickle.load(open(feature_scaler, 'rb'))
    model = pickle.load(open(model, 'rb'))
    
    
    #Open sql
    val_list = []
    validator = open("validation.txt", "r")
    for line in validator:
        val_list.append(line.rstrip())

    mydb = mysql.connector.connect(
      host=val_list[0],
      user=val_list[1],
      passwd=val_list[2],
      database=val_list[3],
      port=val_list[4]
    )
    print("Receiving number interval for zone {0}".format(zone))
        
    mycursor = mydb.cursor()
    sql = """SELECT COUNT(*) as nr FROM team12dublinbus.final where zone = """ +str(zone)+""";"""
    mycursor.execute(sql)
    result_list = []
    rand_no = 0
    for (r) in mycursor:
        rand_no = str(round(row_num/int(r[0]), 4))

    print("Received number interval from DB for zone {0}".format(zone))
    print("Receiving dataframe for zone {0}".format(zone))

    mycursor = mydb.cursor()
    sql = """SELECT `time_to_sec(m.hr_time)`,travel_time,precipIntensity,temperature,humidity,dist,holiday,weekend 
    FROM team12dublinbus.final
    WHERE zone = """+str(zone)+""" and RAND() < """+str(rand_no)+""";"""
    mycursor.execute(sql)

    print("Received dataframe for zone {0}. Processing frame...".format(zone))

    result_list = []
    for (l) in mycursor:
        result_list.extend([l])

    frame = DataFrame.from_records(result_list)
    frame.columns = ['time_of_day', 'time','precipIntensity', 'temperature', 'humidity', 'distance','holiday','weekend']
    
    print("Frame processed. Starting data scaling")
    X= frame[['time_of_day','precipIntensity', 'temperature', 'humidity', 'distance','holiday','weekend']]
    Y = frame[['time']]

    X = feat_scaler.transform(X[['time_of_day','precipIntensity', 'temperature', 'humidity', 'distance','holiday','weekend']])         
    Y = tar_scaler.transform(Y[['time']])  
    
    print("Frames scaled. Starting model prediction")
    predicted = model.predict(X)
    
    print("Starting metric check")
    result = metrics.mean_squared_error(Y,predicted)
    
    result = tar_scaler.inverse_transform(pd.array([1,result]))
    
    print("Completed test for zone {0}. Returning RMSE".format(zone))
    return np.sqrt(result[1])

In [16]:
print(return_test_frame(6))



Receiving number interval for zone 6
Received number interval from DB for zone 6
Receiving dataframe for zone 6
Received dataframe for zone 6. Processing frame...
Frame processed. Starting data scaling
Frames scaled. Starting model prediction
Starting metric check
Completed test for zone 6. Returning RMSE
12.377194614974622


E:\Anaconda\envs\env\lib\site-packages\ipykernel_launcher.py:71: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
E:\Anaconda\envs\env\lib\site-packages\ipykernel_launcher.py:72: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
